In [3]:
# read data 
import json
import pandas as pd
jsonl = []
with open('../data/test.jsonl') as f:
    for i,line in enumerate(f):
#         print(i)
        jsonl.append(json.loads(line.strip()))
df = pd.DataFrame({'jsonl': jsonl})

## Process Data

In [4]:
def fulltext_length(obj):
    text = ' '.join(i for i in obj['section_names'] if i != '__NO_TITLE__') + ' ' + ' '.join(par for sec in obj['sections'] for par in sec)
    return len(text.split())

def fulltext(obj):
    text = ' '.join(par for sec in obj['sections'] for par in sec)
    return text

def fulltext_sec(obj):
    text = [par for sec in obj['sections'] for par in sec]
    return text

def abstract_sec(obj):
    text = [par for sec in obj['abstract_sections'] for par in sec]
    return text

def fullabs(obj):
    text = ' '.join(par for sec in obj['abstract_sections'] for par in sec)
    return text

df['fulltext_length'] = df.jsonl.apply(fulltext_length)
df['fulltext']= df.jsonl.apply(fulltext)
df['fullabs']= df.jsonl.apply(fullabs)
df['fulltext_sec']= df.jsonl.apply(fulltext_sec)

## Example

In [5]:
line = jsonl[0]
# line

In [6]:
text = df.fulltext[0]
# text

In [7]:
abst  = df.fullabs[0]
#abst

## Summarize LSA

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import absolute_import
from __future__ import division, print_function, unicode_literals

from sumy.parsers.html import HtmlParser
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer as Summarizer
from sumy.nlp.stemmers import Stemmer
from sumy.utils import get_stop_words

words=196
def summ_lsa(text):
    LANGUAGE = "english"
    SENTENCES_COUNT = 20
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    
    c = 0
    s = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        if c < words:
            s.append(str(sentence))
            c+= len(str(sentence).split(' '))
        else:
            break
    return " ".join(s)
# summ_lsa(text)
df['lsa_summ']= df.fulltext.apply(summ_lsa)

hyp = df.lsa_summ.tolist()
hyp= [i.replace('\n',' ') for i in hyp]

hyp_path = '../result/lsa.txt'
write_output(hyp,hyp_path)

/home/yued/miniconda3/lib/python3.7/site-packages/sumy/summarizers/lsa.py:76: UserWarning: Number of words (306) is lower than number of sentences (316). LSA algorithm may not work properly.
  warn(message % (words_count, sentences_count))


## Summarize TextRank

In [ ]:
from summa import summarizer
def summ_textrank(text):
    return summarizer.summarize(text, words=196)

In [ ]:
df['textrank_summ']= df.fulltext.apply(summ_textrank)

In [ ]:
df.textrank_summ[1]

## Writting Output

In [ ]:
def write_output(output_list,output_path):
    with open(output_path,'w') as f:
        for i,l in enumerate(output_list):
            f.write(l+'\n')
    print('Write %d lines into %s'%(i,output_path))

In [ ]:
hyp = df.textrank_summ.tolist()
hyp= [i.replace('\n',' ') for i in hyp]
ref = df.fullabs.tolist()
ref_path = '../result/ref.txt'
write_output(ref,ref_path)

## Summarize LexRank

In [114]:
from sumy.summarizers.lex_rank import LexRankSummarizer as Summarizer

words=196
def summ_traditional(text):
    LANGUAGE = "english"
    SENTENCES_COUNT = 20
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    
    c = 0
    s = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        if c < words:
            s.append(str(sentence))
            c+= len(str(sentence).split(' '))
        else:
            break
    return " ".join(s)
# summ_lsa(text)
df['lexrank_summ']= df.fulltext.apply(summ_traditional)

hyp = df.lexrank_summ.tolist()
hyp= [i.replace('\n',' ') for i in hyp]

hyp_path = '../result/lexrank.txt'
write_output(hyp,hyp_path)

Write 5999 lines into ../result/lexrank.txt


## Summarize SumBasics

In [117]:
from sumy.summarizers.sum_basic import SumBasicSummarizer as Summarizer
# summ_lsa(text)
def summ_traditional(text):
    LANGUAGE = "english"
    SENTENCES_COUNT = 20
    parser = PlaintextParser.from_string(text, Tokenizer(LANGUAGE))
    stemmer = Stemmer(LANGUAGE)

    summarizer = Summarizer(stemmer)
    summarizer.stop_words = get_stop_words(LANGUAGE)
    
    c = 0
    s = []
    for sentence in summarizer(parser.document, SENTENCES_COUNT):
        if c < words:
            s.append(str(sentence))
            c+= len(str(sentence).split(' '))
        else:
            break
    return " ".join(s)
df['sumbasic_summ']= df.fulltext.apply(summ_traditional)

hyp = df.sumbasic_summ.tolist()
hyp= [i.replace('\n',' ') for i in hyp]

hyp_path = '../result/sumbasic.txt'
write_output(hyp,hyp_path)

Write 5999 lines into ../result/sumbasic.txt


## Remove empty lines

In [1]:
def remove_empty_lines(hyp_path):
    # replace empty lines
    with open(hyp_path,'r') as f:
        hyps = f.readlines()

    new_hyps=[]
    for idx,l in enumerate(hyps):
        if l == "\n" or len(l.strip())==0:
            new_hyps.append('none')
        else:
            new_hyps.append(l.strip())

    write_output(new_hyps,hyp_path+'.txt')